In [1]:
import pandas as pd
import numpy as np
import glob, os
import xlrd
from collections import defaultdict

In [2]:
#os.chdir("/home/sara/Documents/Immigration/Shared_statistics/")
resident_foreigners_norm = pd.read_table("/home/sara/Documents/Immigration/Shared_statistics/Data_final/resident_foreigners_norm.csv")

province_capitals = sorted(list(set(resident_foreigners_norm["Province"])))

In [3]:
prov_codes = pd.read_table("/home/sara/Documents/Immigration/Data_not_git/Data_distances/Elenco-comuni-italiani.csv", sep= ";", 
                           usecols = ['Codice Regione', 'Codice Città Metropolitana', 
                                      'Codice Comune numerico con 110 province (dal 2010 al 2016)', 
                                      'Denominazione corrente',
                                      'Denominazione regione'])

villasangiovanni_code = prov_codes[(prov_codes["Denominazione corrente"] == "Villa San Giovanni") & (prov_codes["Denominazione regione"] == "Calabria")]["Codice Comune numerico con 110 province (dal 2010 al 2016)"].values[0]
civitavecchia_code = prov_codes[(prov_codes["Denominazione corrente"] == "Civitavecchia") & (prov_codes["Denominazione regione"] == "Lazio")]["Codice Comune numerico con 110 province (dal 2010 al 2016)"].values[0]

# Some territories approximations
prov_codes['Denominazione corrente'].replace(['Verbania', 'Monza', 'Forlì', 'Massa', 'Pesaro', 'Aosta', 'Bolzano', 'Sanluri', 'Olbia', 'Barletta', 'Carbonia', 'Tortolì'], ['Verbano-Cusio-Ossola', 'Monza e della Brianza', 'Forlì-Cesena', 'Massa-Carrara', 'Pesaro e Urbino', "Valle d'Aosta / Vallée d'Aoste", 'Bolzano / Bozen', 'Medio Campidano', 'Olbia-Tempio', 'Barletta-Andria-Trani', 'Carbonia-Iglesias', 'Ogliastra'], inplace = True)

In [4]:
prov_codes.head()

,Codice Regione,Codice Città Metropolitana,Denominazione corrente,Denominazione regione,Codice Comune numerico con 110 province (dal 2010 al 2016)
0,1.0,201,Agliè,Piemonte,1001.0
1,1.0,201,Airasca,Piemonte,1002.0
2,1.0,201,Ala di Stura,Piemonte,1003.0
3,1.0,201,Albiano d'Ivrea,Piemonte,1004.0
4,1.0,201,Alice Superiore,Piemonte,1005.0


In [5]:
prov_codes = prov_codes[prov_codes["Denominazione corrente"].isin(province_capitals)]
prov_codes.index = range(len(prov_codes))

In [6]:
prov_codes.head()

,Codice Regione,Codice Città Metropolitana,Denominazione corrente,Denominazione regione,Codice Comune numerico con 110 province (dal 2010 al 2016)
0,1.0,201,Torino,Piemonte,1272.0
1,1.0,-,Vercelli,Piemonte,2158.0
2,1.0,-,Novara,Piemonte,3106.0
3,1.0,-,Cuneo,Piemonte,4078.0
4,1.0,-,Asti,Piemonte,5005.0


In [7]:
all_codes = prov_codes["Codice Comune numerico con 110 province (dal 2010 al 2016)"].values

In [8]:
name_codes_corr = prov_codes.set_index('Codice Comune numerico con 110 province (dal 2010 al 2016)')['Denominazione corrente'].to_dict()

## About the ISTAT data:

Data Source: http://www.istat.it/it/archivio/157423

ISTAT releases the origin-destination matrices of distances in meters and travel times (in minutes) between all Italian municipalities. The matrices are grouped by Region. The files are provided in text format and all Italian municipalities. 
The islands are treated separately. The matrices of Sicily and Sardinia contain only the between-region distances (the distances only of the municipalities of the regions). 
In a separate excel file are available the distances for the main ports that connect the islands with respect to Peninsular Italy. This allows you to add the travel time by ship to the routes calculated from the main ports of connection to the islands.
 
Once again there are some coherence problems within the different tables:
- the column names are not always the same;
- column are index may be inverted: for all the regions but Lombardia the region-provinces are in the "Destination" column and the other provinces in the "Origin" column. In the Lombardia table the two data are inverted (thus, the need to transpose the matrix).

Morover I am not able to find the Sicilia and Sardinia ports distnces!

In [9]:
# Initialize an empty DataFrame for the distances-matrix
distances = pd.DataFrame()


os.chdir("/home/sara/Documents/Immigration/Data_not_git/Data_distances/Regions")
for file in glob.glob("*.txt"):
    print(file)
    try:
        temp = pd.read_table(file, sep = ";", usecols = ["Origine", "Destinazione", "Total_Mete"])
        temp.rename(columns={'Origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)
    except ValueError:
        temp = pd.read_table(file, sep = ";", usecols = ["origine", "Destinazione", "Total_Mete"])
        temp.rename(columns={'origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)

    for c in temp.columns:
        temp[c] = temp[c].str.replace(",", ".")
        temp[c] = temp[c].astype(float)
        
    if file.split(".")[0] == "Calabria":
        temp_villasangiovanni = temp[(temp["Destination"] == villasangiovanni_code) & (temp["Origin"].isin(all_codes))]
        temp_villasangiovanni = temp_villasangiovanni.replace(name_codes_corr)
        villasangiovanni = temp_villasangiovanni.set_index('Origin')['Distance'].to_dict()
        
    if file.split(".")[0] == "Lazio":
        temp_civitavecchia = temp[(temp["Destination"] == civitavecchia_code) & (temp["Origin"].isin(all_codes))]
        temp_civitavecchia = temp_civitavecchia.replace(name_codes_corr)
        civitavecchia = temp_civitavecchia.set_index('Origin')['Distance'].to_dict()
        
    temp = temp[(temp["Origin"].isin(all_codes)) & (temp["Destination"].isin(all_codes))]
    temp = temp.replace(name_codes_corr)

    temp = temp.pivot(index='Origin', columns='Destination', values='Distance')
    
    if file.split(".")[0] == "Lombardia":
        temp = temp.transpose()
    
    distances = pd.concat([distances, temp], axis=1)

Abruzzo.txt
Basilicata.txt
Calabria.txt
Campania.txt
Emilia.txt
Friuli.txt
Lazio.txt
Liguria.txt
Lombardia.txt
Marche.txt
Molise.txt
Piemonte.txt
Puglia.txt
Sardegna.txt
Sicilia.txt
Toscana.txt
Trentino.txt
Umbria.txt
ValledAosta.txt
Veneto.txt


In [10]:
distances.head()

,Chieti,L'Aquila,Pescara,Teramo,Matera,Potenza,Catanzaro,Cosenza,Crotone,Reggio di Calabria,...,Perugia,Terni,Valle d'Aosta / Vallée d'Aoste,Belluno,Padova,Rovigo,Treviso,Venezia,Verona,Vicenza
Agrigento,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alessandria,617577.83,639953.48,606831.64,586073.06,961908.27,922857.68,1170784.15,1079238.41,1158004.65,1263295.05,...,461149.98,538617.32,167420.83,433768.99,313411.48,320741.23,365648.24,345261.68,235727.45,284071.30
Ancona,166344.52,188720.17,155598.33,134839.76,510674.97,451533.00,804909.01,713363.27,753470.22,897419.92,...,158272.08,183651.48,612167.83,450624.64,325491.73,289541.48,382503.90,362117.34,357362.77,360097.99
Arezzo,380979.42,302482.56,392945.04,359200.33,646521.63,550259.47,798185.94,706640.20,785406.44,890696.85,...,73317.84,166019.12,551609.38,420146.75,295013.84,259063.59,352026.00,331639.44,307940.44,329620.10
Ascoli Piceno,101160.74,93242.70,90414.56,34923.05,445491.19,386349.22,739725.23,648179.49,688286.44,832236.14,...,218542.57,141347.16,721815.27,560272.08,435139.17,399188.92,492151.34,471764.78,467010.21,469745.43


In [11]:
missing = set()
for c in distances.columns:
    if c not in missing:
        missing.update(distances.index[distances[c].isnull()])

In [12]:
missing

{'Agrigento',
 'Cagliari',
 'Caltanissetta',
 'Carbonia-Iglesias',
 'Catania',
 'Enna',
 'Medio Campidano',
 'Messina',
 'Nuoro',
 'Ogliastra',
 'Olbia-Tempio',
 'Oristano',
 'Palermo',
 'Ragusa',
 'Sassari',
 'Siracusa',
 'Trapani'}

To compute the distances between Sicily-Sardinia and the othe Italian regions, it is possible to manually detect (GoogleMaps) some useful distances between the main ports and compute the distances as the sum of multiple component. 

In particular:

from Sicily:
- $d_{(origin province, Trapani)} + d_{(Trapani, Cagliari)} + d_{(Cagliari, destination province)}$ if destination province in Sardinia;
- $d_{(origin province, Messina)} + d_{(Messina, Villa San Giovanni)} + d_{(Villa San Giovanni, destination province)}$ otherwise;

from Sardinia:
- $d_{(origin province, Cagliari)} + d_{(Cagliari, Trapani)} + d_{(Trapani, destination province)}$ if destination province in Sicily;
- $min(d_{(origin province, Cagliari)} + d_{(Cagliari, Trapani)} + d_{(Trapani, Messina)} + d_{(Messina, Villa San Giovanni)} + d_{(Villa San Giovanni, destination province)}, d_{(origin province, Olbia)} + d_{(Olbia, Livorno)} + d_{(Livorno, destination province)}, d_{(origin province, Olbia)} + d_{(Olbia, Civitavecchia)} + d_{(Civitavecchia, destination province)})$ otherwise;

In [13]:
useful = {"Messina-VillaSanGiovanni": 10300, "Cagliari-Trapani": 361000, "Olbia-Civitavecchia": 237000, 
         "Olbia-Livorno": 313000}

In [14]:
distances_approx = distances.copy()
for c in distances_approx.columns:
    index = distances_approx.index[distances_approx[c].isnull()]
    for i in index:
        # Sicily - Sardinia distances
        if (prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sicilia" and prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] == "Sardegna"):
            temp = distances[i]["Trapani"] + useful["Cagliari-Trapani"] + distances["Cagliari"][c]

        '''if prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] == "Sicilia" and prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sardegna":
            print(i, c)
            print(distances[c]["Trapani"] + useful["Cagliari-Trapani"] + distances["Cagliari"][i])'''

        # Sicily - Peninsular Italy
        if (prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sicilia" and prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] != "Sardegna"):
            temp = distances[i]["Messina"] + useful["Messina-VillaSanGiovanni"] + villasangiovanni[c]

        # Sardinia - Peninsular Italy    
        if (prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sardegna" and prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] != "Sicilia"):
            temp = min(distances[i]["Olbia-Tempio"] + useful["Olbia-Civitavecchia"] + civitavecchia[c], 
                       distances[i]["Olbia-Tempio"] + useful["Olbia-Livorno"] + distances["Livorno"][c],
                       distances[i]["Cagliari"] + useful["Cagliari-Trapani"] + distances["Trapani"]["Messina"] + useful["Messina-VillaSanGiovanni"] + villasangiovanni[c])
        distances_approx[c][i] = temp
        distances_approx[i][c] = temp

In [15]:
distances_approx = distances_approx.reindex_axis(sorted(distances_approx.columns), axis=1)
distances_approx.to_csv("/home/sara/Documents/Immigration/Shared_models/Data/Distances_matrix.csv", sep = "\t")

In [16]:
distances_approx.head()

,Agrigento,Alessandria,Ancona,Arezzo,Ascoli Piceno,Asti,Avellino,Bari,Barletta-Andria-Trani,Belluno,...,Udine,Valle d'Aosta / Vallée d'Aoste,Varese,Venezia,Verbano-Cusio-Ossola,Vercelli,Verona,Vibo Valentia,Vicenza,Viterbo
Agrigento,0.00,1520281.89,1154406.75,572277.04,1089222.97,1552214.62,731677.38,703529.24,754750.58,572277.04,...,1576328.45,572277.04,1569915.53,572277.04,1597015.05,1569911.83,572277.04,361969.12,572277.04,1037765.82
Alessandria,1520281.89,0.00,454355.03,382707.95,566168.85,37551.43,810248.50,913327.98,860796.79,433768.99,...,455852.09,167420.83,141987.21,345261.68,141350.35,55036.19,235727.45,1173507.81,284071.30,523140.35
Ancona,1154406.75,461092.48,0.00,252205.19,113923.38,493407.15,427977.88,461082.51,408551.31,450624.64,...,472707.74,612167.83,489332.51,362117.34,514470.78,510891.92,357362.77,807632.68,360097.99,256977.71
Arezzo,572277.04,388784.63,230345.50,0.00,285842.76,421530.33,437650.29,631732.33,579201.14,420146.75,...,442229.85,551609.38,438191.62,331639.44,465048.45,439224.75,307940.44,800909.60,329620.10,150542.14
Ascoli Piceno,1089222.97,570739.92,114437.18,285809.70,0.00,603054.59,362794.10,393785.23,341254.03,560272.08,...,582355.18,721815.27,596190.38,471764.78,624118.22,620539.36,467010.21,742448.90,469745.43,198048.14
